<a href="http://landlab.github.io"><img style="float: left" src="../../../landlab_header.png"></a>

# Using the ChiFinder Component

The `ChiFinder` component creates a map of the $\chi$ drainage network index from a digital elevation model. The $\chi$ index, described by Perron and Royden (2013), is a function of drainage area, $A$, and elevation, $\eta$:

$$\chi = \int\limits_{x_b}^{x} \left(\frac{A_0}{A(x)}\right)^\theta dx$$

where $x_b$ is the location of the outlet of a watershed of interest, $x$ is a position on a channel somewhere upstream, $A_0$ is a drainage area scale, and $\theta$ is the concavity index parameter, often taken to be $\approx$0.5.

This tutorial shows briefly how to use the `ChiFinder` on natural or synthetic data.

## Imports and inline docs

First, import what we'll need:

In [ ]:
import copy

import matplotlib as mpl
import numpy as np

from landlab import RasterModelGrid, imshow_grid
from landlab.components import ChiFinder, FlowAccumulator
from landlab.io import read_esri_ascii

The docstring describes the component and provides some simple examples:

In [ ]:
print(ChiFinder.__doc__)

## Example

In this example, we read in a small digital elevation model (DEM) from NASADEM for an area on the Colorado high plains (USA) that includes a portion of an escarpment along the west side of a drainage known as West Bijou Creek (see Rengers & Tucker, 2014).

The DEM file is in ESRI Ascii format, but is in a geographic projection, with horizontal units of decimal degrees. To calculate slope gradients properly, we'll first read the DEM into a Landlab grid object that has this geographic projec. Then we'll create a second grid with 30 m cell spacing (approximately equal to the NASADEM's resolution), and copy the elevation field from the geographic DEM. This isn't a proper projection of course, but it will do for purposes of this example.

In [ ]:
# read the DEM
(grid_geog, elev) = read_esri_ascii("west_bijou_escarpment_snippet.asc")
grid = RasterModelGrid(
    (grid_geog.number_of_node_rows, grid_geog.number_of_node_columns), xy_spacing=30.0
)
grid.add_field("topographic__elevation", elev, at="node")

In [ ]:
cmap = copy.copy(mpl.colormaps["pink"])
imshow_grid(grid, elev, cmap=cmap, colorbar_label="Elevation (m)")

The `ChiFinder` needs to have drainage areas pre-calculated. We'll do that with the `FlowAccumulator` component. We'll have the component do D8 flow routing (each DEM cell drains to whichever of its 8 neighbors lies in the steepest downslope direction), and fill pits (depressions in the DEM that would otherwise block the flow) using the `LakeMapperBarnes` component. The latter two arguments below tell the lake mapper to update the flow directions and drainage areas after filling the pits.

In [ ]:
fa = FlowAccumulator(
    grid,
    flow_director="FlowDirectorD8",  # use D8 routing
    depression_finder="LakeMapperBarnes",  # pit filler
    method="D8",  # pit filler use D8 too
    redirect_flow_steepest_descent=True,  # re-calculate flow dirs
    reaccumulate_flow=True,  # re-calculate drainagea area
)
fa.run_one_step()  # run the flow accumulator
cmap = copy.copy(mpl.colormaps["Blues"])
imshow_grid(
    grid,
    np.log10(grid.at_node["drainage_area"] + 1.0),  # sq root helps show drainage
    cmap=cmap,
    colorbar_label="Log10(drainage area (m2))",
)

Now run the `ChiFinder` and display the map of $\chi$ values:

In [ ]:
cf = ChiFinder(
    grid,
    reference_concavity=0.5,
    min_drainage_area=1.0,
    clobber=True,
)
cf.calculate_chi()
cmap = copy.copy(mpl.colormaps["viridis"])
imshow_grid(
    grid,
    grid.at_node["channel__chi_index"],
    cmap=cmap,
    colorbar_label="Chi index",
)

## References

Perron, J., Royden, L. (2012). An integral approach to bedrock river
profile analysis Earth Surface Processes and Landforms  38(6), 570-576. https://dx.doi.org/10.1002/esp.3302
    
Rengers, F. K., & Tucker, G. E. (2014). Analysis and modeling of gully headcut dynamics, North American high plains. Journal of Geophysical Research: Earth Surface, 119(5), 983-1003. https://doi.org/10.1002/2013JF002962

For more Landlab tutorials, click here: https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html